In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("curl -o coefficients.csv https://static-content.springer.com/esm/art%3A10.1186%2Fs13059-016-1068-z/MediaObjects/13059_2016_1068_MOESM3_ESM.csv")
os.system("curl -o reference_feature_values.csv https://static-content.springer.com/esm/art%3A10.1186%2Fgb-2013-14-10-r115/MediaObjects/13059_2013_3156_MOESM22_ESM.csv")

0

In [3]:
df = pd.read_csv('coefficients.csv')

df['feature'] = df['CpGmarker']
df['coefficient'] = df['CoefficientTraining']

In [4]:
features = df['feature'][1:].tolist()

weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

In [5]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=148, out_features=1, bias=True)
)

In [7]:
reference_feature_values = pd.read_csv('reference_feature_values.csv', index_col=0)
reference_feature_values = reference_feature_values.loc[features]
reference_feature_values = reference_feature_values['goldstandard2'].tolist()

In [8]:
weights_dict = {
    'reference_feature_values': reference_feature_values,
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
    'model_class': 'LinearModel',
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2016,
    'implementation_approved_by_author(s)': '⌛',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Knight, Anna K., et al. \"An epigenetic clock for gestational age at birth based on blood methylation data.\" Genome biology 17.1 (2016): 1-11.",
    'doi': "https://doi.org/10.1186/s13059-016-1068-z",
    "notes": None,
}

In [9]:
torch.save(weights_dict, '../weights/knight.pt')
torch.save(metadata_dict, '../metadata/knight.pt')

In [10]:
os.system("rm coefficients.csv")
os.system("rm reference_feature_values.csv")

0